![Texto alternativo](https://cdn-ilbdkgn.nitrocdn.com/KyPgQQQAVfBmOoLeBzkzAekksAGOgYiS/assets/images/optimized/rev-507152b/www.modernheartandvascular.com/wp-content/uploads/2022/10/HEart-failure.png)

El objetivo de este proyecto es construir un modelo de Machine Learning que pueda predecir si un paciente tiene una enfermedad cardíaca basándonos en sus características médicas.

## Cargar librerias y Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score


# Cargar los datos
df = pd.read_csv("./data/heart.csv")

# Manejo de desbalanceo con SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
            

## Estructura del Dataset

In [ ]:
# Ver primeras filas
print(df.head())

# Información general
print(df.info())

# Resumen estadístico
print(df.describe())


Nos encontramos ante un dataset con 1025 filas, 14 columnas.
De los cuales, 5 variables son numéricas (int64 o float64).
8 variables son categóricas (object) y la variable objetivo (target) es binaria: 0 = No enfermedad, 1 = Enfermedad.


•	Edad (age)

•	Sexo (sex)

•	Presión arterial en reposo (trestbps)

•	Colesterol (chol)

•	Frecuencia cardíaca máxima alcanzada (thalach)

•	Angina inducida por ejercicio (exang)

•	Depresión del ST inducida por el ejercicio (oldpeak)

•	Tipo de dolor en el pecho (cp - diferentes tipos de dolor)

•	Resultados de electrocardiograma en reposo (restecg)

Se observa:
Varias columnas categóricas (sex, chest_pain_type, etc.) deben ser convertidas a variables numéricas.
La columna cholestoral parece tener valores altos (máx: 564), posible outlier.
La columna vessels_colored_by_flourosopy tiene valores en texto que podrían requerir limpieza.

In [ ]:
# Convertir columnas categóricas a numéricas
categorical_mappings = {
    'sex': {'Male': 1, 'Female': 0},
    'chest_pain_type': {
        'Typical angina': 0, 'Atypical angina': 1, 'Non-anginal pain': 2, 'Asymptomatic': 3
    },
    'fasting_blood_sugar': {'Lower than 120 mg/ml': 0, 'Greater than 120 mg/ml': 1},
    'rest_ecg': {'Normal': 0, 'ST-T wave abnormality': 1, 'Left ventricular hypertrophy': 2},
    'exercise_induced_angina': {'No': 0, 'Yes': 1},
    'slope': {'Upsloping': 0, 'Flat': 1, 'Downsloping': 2},
    'vessels_colored_by_flourosopy': {'Zero': 0, 'One': 1, 'Two': 2, 'Three': 3, 'Four': 4},
    'thalassemia': {'Normal': 0, 'Fixed Defect': 1, 'Reversable Defect': 2}
}

# Aplicar las conversiones
for col, mapping in categorical_mappings.items():
    df[col] = df[col].map(mapping)

# Manejo de outliers en 'cholestoral' (usando percentil 99 para detectar valores extremos)
cholesterol_threshold = np.percentile(df['cholestoral'], 99)
df = df[df['cholestoral'] <= cholesterol_threshold]

# Mostrar resultados
df.info(), df.head()

Faltan algunos valores en thalassemia (7 valores nulos), procedemos a eliminarlo ya que nos pocos (7 frente a  1016)

In [ ]:
# Eliminar filas con valores nulos en 'thalassemia'
df = df.dropna(subset=['thalassemia'])

# Verificar que no hay valores nulos
df.info()


## MINI EDA

Distribución de la Variable objetico

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x=df["target"], palette="coolwarm")
plt.title("Distribución de Pacientes con Enfermedad Cardíaca")
plt.show()


Tenemos una variable target balanceada

Correlación entre variabbles

In [ ]:
# Usamos LabelEncoder para transformar las categorías en números
encoder = LabelEncoder()
for col in categorical_mappings:
    df[col] = encoder.fit_transform(df[col])

In [ ]:
# Mapa de calor de correlaciones
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Mapa de Correlación entre Variables")
plt.show()


De este análisis se puede concluir:

Max_heart_rate y target tienen correlación positiva (0.40), lo que indica que una mayor frecuencia cardíaca máxima puede estar asociada con la enfermedad.

Oldpeak (Depresión del ST) tiene correlación negativa (-0.57) con target, lo que sugiere que valores más altos pueden indicar menor riesgo.

Chest_pain_type tiene correlación positiva fuerte (0.43) con la variable target, lo que indica que el tipo de dolor en el pecho es un buen predictor.

Algunas variables (cholestoral, resting_blood_pressure) no parecen estar fuertemente correlacionadas con target, por lo que podrían no ser tan relevantes.

 Distribución de Pacientes según Edad y Género

In [ ]:
plt.figure(figsize=(12,5))
sns.histplot(df["age"], bins=20, kde=True, color="blue")
plt.title("Distribución de Edad de los Pacientes")
plt.show()

plt.figure(figsize=(6,4))
sns.countplot(x=df["sex"], hue=df["target"], palette="viridis")
plt.xticks([0, 1], ["Mujeres", "Hombres"])
plt.title("Distribución por Género y Enfermedad")
plt.show()


De este análisis se concluye que la mayoría de los pacientes con enfermedad son mayores de cierta edad por lo que la edad puede ser un factor clave para padecer la enfermedad de estudio.

Las mujeres tienen menor riesgo de padecer la enfermedad que los hombres.


## Revisión de datos

 Manejo de Datos Categóricos
Algunas variables en el dataset son categóricas y deben ser convertidas en formato numérico si aún no lo están.

In [ ]:
# Convertir variables categóricas en numéricas
categorical_cols = ['sex', 'chest_pain_type', 'fasting_blood_sugar', 'rest_ecg', 
                    'exercise_induced_angina', 'slope', 'vessels_colored_by_flourosopy', 'thalassemia']



Normalización de Variables Numéricas

Para mejorar el desempeño de algunos modelos, escalamos los datos con StandardScaler.

In [ ]:
# Normalizar variables numéricas:
scaler = StandardScaler()
numerical_cols = ['age', 'resting_blood_pressure', 'cholestoral', 'Max_heart_rate', 'oldpeak']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])



Ahora dividimos es dataset en Train/Test para tener los datos en conjuntos de entrenamiento y de prueba.

In [ ]:
# Separar variables predictoras y objetivo
X = df.drop("target", axis=1)
y = df["target"]

# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar tamaño de los conjuntos
X_train.shape, X_test.shape


# Manejo de desbalanceo con SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
            

Estudiemos los modelos que presenten mayor rendimiento.

In [ ]:

# Optimización de hiperparámetros con RandomizedSearchCV
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rf, param_distributions=param_grid, n_iter=10, cv=5, scoring="roc_auc", n_jobs=-1)
random_search.fit(X_train, y_train)

# Mejor modelo
best_model = random_search.best_estimator_
            

La Regresión Logística tiene un 85.33% precisión promedio ofreciendo un rendimiento sólido y generalizable,
mientras que el Random Forest un 99.70% de precisión promedio.Esto ofrece una precisión altísima, pero esto sugiere que podría estar sobreajustando a los datos.

## Pasos a seguir

Vamos a optimizar el modelo de Random Forest ajustando los hiperparámetros con RandomizedSearchCV para mejorar generalización.


In [ ]:

# Optimización de hiperparámetros con RandomizedSearchCV
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rf, param_distributions=param_grid, n_iter=10, cv=5, scoring="roc_auc", n_jobs=-1)
random_search.fit(X_train, y_train)

# Mejor modelo
best_model = random_search.best_estimator_
            

El mejor puntaje de precisión obtenido en la validación cruzada es 0.9901, esto nos da información de que el modelo tiene un desempeño excelente en los datos de entrenamiento con estos hiperparámetros.

Veamos si el modelo generaliza bien o sigue sobreajustando.

In [ ]:

# Optimización de hiperparámetros con RandomizedSearchCV
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rf, param_distributions=param_grid, n_iter=10, cv=5, scoring="roc_auc", n_jobs=-1)
random_search.fit(X_train, y_train)

# Mejor modelo
best_model = random_search.best_estimator_
            

Las variables más relevantes en la predicción son aquellas relacionadas con la salud cardíaca directamente, como la fluoroscopia, el ritmo cardíaco, el tipo de dolor en el pecho, la talasemia y el "oldpeak".

Las variables menos importantes son aquellas relacionadas con la fisiología (como el sexo) o con medidas que, en este conjunto de datos específico, no tienen un gran impacto predictivo (como el azúcar en sangre en ayunas).

Las variables relacionadas con los síntomas y condiciones más específicas de problemas cardíacos parecen ser las que aportan más valor a la predicción, mientras que las variables más generales o relacionadas con condiciones menos directamente relacionadas con problemas cardíacos tienen menor importancia.

Podemos eliminar las variables menos relevantes para probar si el modelo mejora.


In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred_best_rf = best_rf.predict(X_test)

# Evaluar el modelo optimizado
acc_best_rf = accuracy_score(y_test, y_pred_best_rf)
report_best_rf = classification_report(y_test, y_pred_best_rf)

acc_best_rf, report_best_rf

# Explicabilidad del modelo con SHAP
import shap

explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values[1], X_test)
            

Precisión del modelo optimizado en el conjunto de prueba: 98,51%

Todas las métricas son cercanas a 1


Eliminamos fasting_blood_sugar, rest_ecg y sex, que tienen poca importancia.
Reentrenamos el modelo para ver si mantenemos un buen rendimiento con menos complejidad.

In [ ]:
# Eliminar las variables menos relevantes
cols_to_drop = ["fasting_blood_sugar", "rest_ecg", "sex"]
X_reduced = X.drop(columns=cols_to_drop)

# Dividir en entrenamiento y prueba nuevamente
X_train_red, X_test_red, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42, stratify=y)

# Reentrenar Random Forest con el nuevo conjunto reducido
best_rf.fit(X_train_red, y_train)

# Evaluar en el conjunto de prueba
y_pred_reduced = best_rf.predict(X_test_red)
acc_reduced = accuracy_score(y_test, y_pred_reduced)
report_reduced = classification_report(y_test, y_pred_reduced)

acc_reduced, report_reduced

# Manejo de desbalanceo con SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
            

Reducir las variables ha mejorado el rendimiento (97%) lo que significa que el modelo puede ser más simple sin perder precisión.

Veamos ahora XGBoost, que es un modelo potente que maneja bien datos tabulares y puede reducir el sobreajuste.

In [ ]:
from xgboost import XGBClassifier

# Definir y entrenar el modelo XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb_model.fit(X_train_red, y_train)

# Evaluar en el conjunto de prueba
y_pred_xgb = xgb_model.predict(X_test_red)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
report_xgb = classification_report(y_test, y_pred_xgb)

acc_xgb, report_xgb

# Explicabilidad del modelo con SHAP
import shap

explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values[1], X_test)
            

Este modelo tiene resultados parecidos